# Preprocess the data
A mesh is generated for each image to be used afterwards.

In [1]:
import os
from os.path import expanduser as eu
from headctools.registration import register_file_folder
from headctools.preprocessing import ct_skull

imgs = eu('~/Code/datasets/cq500/converted/selected/')
fixed_img = os.path.join(imgs, 'CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz')
# Registered images will be saved in a sub folder with the name of the fixed image

params = {
    'reg_atlas_path': fixed_img
}

ct_skull(imgs, params = params)

KeyboardInterrupt: 